In [0]:
!pip install catboost

     |████████████████████████████████| 62.9MB 149kB/s 


In [0]:
# Libraries
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime
import lightgbm as lgb
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import os
import pickle
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn import metrics
import json
import ast
import time
from sklearn import linear_model
from tqdm import tqdm_notebook
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_selection import GenericUnivariateSelect, SelectPercentile, SelectKBest, f_classif, mutual_info_classif, RFE
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from catboost import CatBoostClassifier

import json
from IPython.display import HTML

from plotly import tools
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import warnings
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



In [0]:
PATH_FILE = '/content/'
train = pd.read_csv(os.path.join(PATH_FILE, 'train_features.csv'), index_col='match_id_hash')
target = pd.read_csv(os.path.join(PATH_FILE, 'train_targets.csv'), index_col='match_id_hash')
test = pd.read_csv(os.path.join(PATH_FILE, 'test_features.csv'), index_col='match_id_hash')

In [0]:
train.describe(include='all')

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
train_add_features, test_ctb = pd.read_pickle(os.path.join(PATH_FILE, 'train.pkl')),\
                               pd.read_pickle(os.path.join(PATH_FILE, 'test.pkl'))

In [0]:
pca_train, pca_test = pd.read_pickle(os.path.join(PATH_FILE, 'train_pca_wo_emb.pkl')),\
                      pd.read_pickle(os.path.join(PATH_FILE, 'test_pca_wo_emb.pkl'))

pca_train.set_index(train.index.values, inplace=True)
pca_test.set_index(test.index.values, inplace=True)


In [0]:
pca_test.sample()

,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,pca_21,pca_22,pca_23,pca_24,pca_25,pca_26,pca_27,pca_28,pca_29,pca_30,pca_31,pca_32,pca_33,pca_34,pca_35,pca_36,pca_37,pca_38,pca_39,pca_40,pca_41,pca_42,pca_43,pca_44,pca_45,pca_46,pca_47,pca_48,pca_49,pca_50,pca_51,pca_52,pca_53,pca_54,pca_55,pca_56,pca_57,pca_58,pca_59,pca_60,pca_61,pca_62,pca_63,pca_64,pca_65,pca_66,pca_67,pca_68,pca_69,pca_70,pca_71,pca_72,pca_73,pca_74,pca_75,pca_76,pca_77,pca_78,pca_79,pca_80,pca_81,pca_82,pca_83,pca_84,pca_85,pca_86,pca_87,pca_88,pca_89,pca_90,pca_91,pca_92,pca_93,pca_94,pca_95,pca_96,pca_97,pca_98,pca_99,pca_100,pca_101,pca_102,pca_103,pca_104,pca_105,pca_106,pca_107,pca_108,pca_109,pca_110,pca_111,pca_112,pca_113,pca_114,pca_115,pca_116,pca_117,pca_118,pca_119,pca_120,pca_121,pca_122,pca_123,pca_124,pca_125,pca_126,pca_127,pca_128,pca_129,pca_130,pca_131,pca_132,pca_133,pca_134,pca_135,pca_136,pca_137,pca_138,pca_139,pca_140,pca_141,pca_142,pca_143,pca_144,pca_145,pca_146,pca_147,pca_148,pca_149,pca_150,pca_151,pca_152,pca_153,pca_154,pca_155,pca_156,pca_157,pca_158,pca_159,pca_160,pca_161,pca_162,pca_163,pca_164,pca_165,pca_166,pca_167,pca_168,pca_169,pca_170,pca_171,pca_172,pca_173,pca_174,pca_175,pca_176,pca_177,pca_178,pca_179,pca_180,pca_181,pca_182,pca_183,pca_184,pca_185,pca_186,pca_187,pca_188,pca_189,pca_190,pca_191,pca_192,pca_193,pca_194,pca_195,pca_196,pca_197,pca_198,pca_199,pca_200,pca_201,pca_202,pca_203,pca_204,pca_205,pca_206,pca_207,pca_208,pca_209,pca_210,pca_211,pca_212,pca_213,pca_214,pca_215,pca_216,pca_217,pca_218,pca_219,pca_220,pca_221,pca_222,pca_223,pca_224,pca_225,pca_226,pca_227,pca_228,pca_229,pca_230,pca_231,pca_232,pca_233,pca_234,pca_235,pca_236,pca_237,pca_238,pca_239,pca_240,pca_241,pca_242,pca_243,pca_244,pca_245,pca_246,pca_247,pca_248,pca_249,pca_250,pca_251,pca_252,pca_253,pca_254,pca_255,pca_256
a39d40bbb9521373d1c01ed6297a114e,26736.317045,-5644.92126,2358.353586,2080.040325,-292.81583,-269.622235,-435.835848,345.505231,16.374735,-151.481829,-65.546772,117.507549,-113.450331,4.589661,-18.809388,33.492162,63.267416,-10.817294,-87.393643,-74.007283,-33.536254,50.260496,-114.556818,2.406927,-6.170853,-3.913356,-7.685653,-7.813708,-13.562036,9.003613,-44.964399,41.401321,-32.003958,-2.973503,1.049892,-0.821599,-2.816784,0.350279,-4.397612,11.926839,-9.157705,3.261729,5.250087,4.720066,75.796507,-1.756004,-0.717236,-0.802552,1.114006,-1.001672,-0.915698,1.199617,-0.198145,0.25032,0.199447,0.155763,-0.02815,0.099143,0.133136,-0.100705,-0.020611,-0.386792,-0.112644,-0.093629,-0.648439,-0.879385,0.15393,0.828631,0.243891,0.925514,0.219758,0.039447,0.838352,0.318623,0.088054,-0.248767,0.538074,0.204204,0.198157,0.215529,-0.129324,-0.387556,0.908648,0.160572,-0.188254,0.073808,0.601205,0.315979,-0.172247,-0.191593,-0.170318,-0.033678,0.416388,0.353027,-0.297613,-0.7123,0.235601,0.184378,-0.481134,-0.296334,0.054812,0.135479,0.159329,-0.076447,-0.020937,-0.021897,-0.130214,0.181417,-0.109079,-0.065642,-0.011208,0.048857,-0.036446,0.106699,0.078708,0.017732,0.077191,0.020806,-0.007582,-0.003943,0.006562,-0.111795,0.130178,-0.120945,-0.039155,0.002745,-0.022617,0.027274,0.004528,0.003412,0.092719,-0.024363,0.081296,-0.044382,-0.031988,0.078205,0.011269,-0.078937,0.069484,-0.035189,-0.080439,0.075184,-0.136102,0.020212,-0.096874,0.06021,-0.039904,-0.037372,-0.002024,0.054865,-0.101148,0.003114,0.053578,-0.003243,-0.027208,-0.023207,-0.045905,0.026919,0.033962,0.110646,0.003921,-0.064932,0.072868,-0.01725,0.012,-0.000355,0.079672,-0.053786,-0.07704,0.08321,-0.121146,-0.072552,0.954673,0.105711,0.021017,-0.036777,-0.015112,-0.01507,0.42683,-0.143879,-0.103176,-0.00027,-0.012975,0.013885,0.026722,-0.042435,-0.024784,-0.023003,-0.022221,0.002931,0.016049,-0.053589,-0.016506,-0.024805,-0.027387,-0.017514,-0.048816,0.018488,0.020046,-0.027556,-0.037354,-0.009079,0.002397,0.015123,0.031856,-0.012311,0.017829,0.012204,0.013942,0.015477

In [0]:
train_add_features, test_ctb = train_add_features.join(pca_train), \
                               test_ctb.join(pca_test)

In [0]:
test_ctb.shape, train_add_features.shape

((10000, 479), (39675, 479))

In [0]:
test_ctb.sample()

,last_tower_radiant,r_barack_damage,d_barack_damage,r_tower_damage,d_tower_damage,r_total_kills,d_total_kills,total_kills_ratio,r_std_kills,d_std_kills,std_kills_ratio,r_mean_kills,d_mean_kills,mean_kills_ratio,r_total_deaths,d_total_deaths,total_deaths_ratio,r_std_deaths,d_std_deaths,std_deaths_ratio,r_mean_deaths,d_mean_deaths,mean_deaths_ratio,r_total_assists,d_total_assists,total_assists_ratio,r_std_assists,d_std_assists,std_assists_ratio,r_mean_assists,d_mean_assists,mean_assists_ratio,r_total_denies,d_total_denies,total_denies_ratio,r_std_denies,d_std_denies,std_denies_ratio,r_mean_denies,d_mean_denies,mean_denies_ratio,r_total_gold,d_total_gold,total_gold_ratio,r_std_gold,d_std_gold,std_gold_ratio,r_mean_gold,d_mean_gold,mean_gold_ratio,r_total_lh,d_total_lh,total_lh_ratio,r_std_lh,d_std_lh,std_lh_ratio,r_mean_lh,d_mean_lh,mean_lh_ratio,r_total_xp,d_total_xp,total_xp_ratio,r_std_xp,d_std_xp,std_xp_ratio,r_mean_xp,d_mean_xp,mean_xp_ratio,r_total_health,d_total_health,total_health_ratio,r_std_health,d_std_health,std_health_ratio,r_mean_health,d_mean_health,mean_health_ratio,r_total_max_health,d_total_max_health,total_max_health_ratio,r_std_max_health,d_std_max_health,std_max_health_ratio,r_mean_max_health,d_mean_max_health,mean_max_health_ratio,r_total_max_mana,d_total_max_mana,total_max_mana_ratio,r_std_max_mana,d_std_max_mana,std_max_mana_ratio,r_mean_max_mana,d_mean_max_mana,mean_max_mana_ratio,r_total_level,d_total_level,total_level_ratio,r_std_level,d_std_level,std_level_ratio,r_mean_level,d_mean_level,mean_level_ratio,r_std_x,d_std_x,std_x_ratio,r_mean_x,d_mean_x,mean_x_ratio,r_std_y,d_std_y,std_y_ratio,r_mean_y,d_mean_y,mean_y_ratio,r_total_stuns,d_total_stuns,total_stuns_ratio,r_std_stuns,d_std_stuns,std_stuns_ratio,r_mean_stuns,d_mean_stuns,mean_stuns_ratio,r_total_creeps_stacked,d_total_creeps_stacked,total_creeps_stacked_ratio,r_std_creeps_stacked,d_std_creeps_stacked,std_creeps_stacked_ratio,r_mean_creeps_stacked,d_mean_creeps_stacked,r_total_camps_stacked,r_std_camps_stacked,d_std_camps_stacked,std_camps_stacked_ratio,d_mean_camps_stacked,r_total_rune_pickups,d_total_rune_pickups,total_rune_pickups_ratio,r_std_rune_pickups,d_std_rune_pickups,std_rune_pickups_ratio,r_mean_rune_pickups,d_mean_rune_pickups,mean_rune_pickups_ratio,d_total_firstblood_claimed,total_firstblood_claimed_ratio,r_std_firstblood_claimed,d_std_firstblood_claimed,std_firstblood_claimed_ratio,d_mean_firstblood_claimed,r_total_teamfight_participation,d_total_teamfight_participation,total_teamfight_participation_ratio,r_std_teamfight_participation,d_std_teamfight_participation,std_teamfight_participation_ratio,r_mean_teamfight_participation,d_mean_teamfight_participation,mean_teamfight_participation_ratio,r_total_towers_killed,d_total_towers_killed,total_towers_killed_ratio,r_std_towers_killed,d_std_towers_killed,std_towers_killed_ratio,r_mean_towers_killed,d_mean_towers_killed,mean_towers_killed_ratio,r_total_roshans_killed,d_total_roshans_killed,total_roshans_killed_ratio,r_std_roshans_killed,d_std_roshans_killed,std_roshans_killed_ratio,r_mean_roshans_killed,d_mean_roshans_killed,mean_roshans_killed_ratio,r_total_obs_placed,d_total_obs_placed,total_obs_placed_ratio,r_std_obs_placed,d_std_obs_placed,std_obs_placed_ratio,r_mean_obs_placed,d_mean_obs_placed,mean_obs_placed_ratio,r_total_sen_placed,d_total_sen_placed,total_sen_placed_ratio,r_std_sen_placed,d_std_sen_placed,std_sen_placed_ratio,r_mean_sen_placed,d_mean_sen_placed,mean_sen_placed_ratio,r_total_kda,d_total_kda,total_kda_ratio,r_std_kda,d_std_kda,std_kda_ratio,r_mean_kda,d_mean_kda,mean_kda_ratio,r_total_towers_killed_thr,barack_damage_ratio,tower_damage_ratio,r_total_towers,r_total_damage_given,r_total_damage_taken,r_total_killed,d_total_towers,d_total_damage_given,d_total_damage_taken,d_total_killed,r_total_damage_ratio,d_total_damage_ratio,damage_ratio,kmeans_on_all,d_total_towers_killed_thr,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_1

#### Собираем данные

In [0]:
def join_dataset(pickle_files):
  for f in pickle_files:
    with open(f, 'rb') as cur_f:
      if f.find('aggregates.pkl') != -1:
        train_add_features, test_ctb = pickle.load(cur_f)
        continue
      elif f.find('test') != -1:
        test_ctb = test_ctb.join(pickle.load(cur_f))
      elif f.find('train') != -1:
        train_add_features = train_add_features.join(pickle.load(cur_f))

  train_add_features.fillna(0, inplace=True)
  train_add_features.replace(np.inf, 10000, inplace=True)

  test_ctb.fillna(0, inplace=True)
  test_ctb.replace(np.inf, 10000, inplace=True)

  return [train_add_features, test_ctb]

In [0]:
files = [os.path.join(PATH_FILE, 'aggregates.pkl'),
        #  os.path.join(PATH_FILE, 'test_clusters.pkl'), os.path.join(PATH_FILE, 'train_clusters.pkl'),
        #  os.path.join(PATH_FILE, 'test_dist.pkl'), os.path.join(PATH_FILE, 'train_dist.pkl'),
        #  os.path.join(PATH_FILE, 'test_agg_w_per_minute.pkl'), os.path.join(PATH_FILE, 'train_agg_w_per_minute.pkl'),
         os.path.join(PATH_FILE, 'test_last_tower.pkl'), os.path.join(PATH_FILE, 'train_last_tower.pkl'),]

In [0]:
train_add_features, test_ctb = join_dataset(files)

In [0]:
list(enumerate(test_ctb))

In [0]:
train_add_features.shape, test_ctb.shape

((39675, 212), (10000, 212))

In [0]:
train_add_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39675 entries, a400b8f29dece5f4d266f49f1ae2e98a to 9928dfde50efcbdb2055da23dcdbc101
Columns: 252 entries, last_tower_radiant to d5_cluster
dtypes: float64(180), int32(1), int64(49), object(22)
memory usage: 76.4+ MB


#### Adding Features

In [0]:
# train_add_features['r_total_towers_killed_thr'] = train_add_features['r_total_towers_killed'] > 3
# test_ctb['r_total_towers_killed_thr'] = test_ctb['r_total_towers_killed'] > 3

train_add_features['d_total_towers_killed_thr'] = train_add_features['d_total_towers_killed'] > 3
test_ctb['d_total_towers_killed_thr'] = test_ctb['d_total_towers_killed'] > 3

train_add_features['last_tower_radiant'] = train_add_features['last_tower_radiant'].astype(bool)
test_ctb['last_tower_radiant'] = test_ctb['last_tower_radiant'].astype(bool)

# test_ctb['game_time_cat'] = test['game_time'] // 60
# train_add_features['game_time_cat'] = train['game_time'] // 60

### Try logit

#### simple RandomForest

In [0]:
%%time
model = RandomForestClassifier(n_estimators=100, n_jobs=4, random_state=17)
model.fit(X_train, y_train)

CPU times: user 16.4 s, sys: 0 ns, total: 16.4 s
Wall time: 8.4 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                       oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [0]:
feat_import = pd.DataFrame(model.feature_importances_, index=train.columns).sort_values(by=0, ascending=False).head(30)

In [0]:
feat_import

In [0]:
y_pred = model.predict_proba(X_valid)[:, 1]

In [0]:
from sklearn.metrics import roc_auc_score

valid_score = roc_auc_score(y_valid, y_pred)
print('Validation ROC-AUC score:', valid_score)

Validation ROC-AUC score: 0.780089361051384


Out if curiosiry, we can calculate accuracy of a classifier which predicts class 1 if predicted probability is higher than 50%.

In [0]:
from sklearn.metrics import accuracy_score

valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
print('Validation accuracy of P>0.5 classifier:', valid_accuracy)

Validation accuracy of P>0.5 classifier: 0.7007477106611778


In [0]:
y_pred

array([0.32, 0.36, 0.62, ..., 0.36, 0.31, 0.47])

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(train_add_features, 
                                                      target['radiant_win'].values, 
                                                      test_size=0.3, 
                                                      random_state=23)

### Logit

In [0]:
0 last_tower_radiant object
207 r_total_towers_killed_thr object
221 kmeans_on_all int32
222 d_total_towers_killed_thr object

In [0]:
train_add_features = train_add_features.join(pd.get_dummies(train_add_features['d_total_towers_killed_thr'], columns=['d_towers_thr']).astype(int))
test_ctb = test_ctb.join(pd.get_dummies(test_ctb['d_total_towers_killed_thr'],columns=['d_towers_thr']).astype(int))

train_add_features = train_add_features.join(pd.get_dummies(train_add_features['r_total_towers_killed_thr'], columns=['r_towers_thr']))
test_ctb = test_ctb.join(pd.get_dummies(test_ctb['r_total_towers_killed_thr'], columns=['r_towers_thr']))

train_add_features = train_add_features.join(pd.get_dummies(train_add_features['last_tower_radiant'], columns=['last_tower_radiant']))
test_ctb = test_ctb.join(pd.get_dummies(test_ctb['last_tower_radiant'], columns=['last_tower_radiant']))


train_add_features = train_add_features.join(pd.get_dummies(train_add_features['kmeans_on_all'], columns=['last_tower_radiant']))
test_ctb = test_ctb.join(pd.get_dummies(test_ctb['kmeans_on_all'], columns=['kmeans_on_all']))


train_add_features.drop(columds=['d_total_towers_killed_thr', 'r_total_towers_killed_thr',
                                 'kmeans_on_all', 'last_tower_radiane'], inlace=True)

In [0]:
logit = LinearRegression()
logit.fit(X_train, y_train)
y_pred_logit = logit.predict_proba(X_valid)[:, 1]

In [0]:
valid_score_logit = roc_auc_score(y_valid, y_pred_logit)
print('Validation ROC-AUC score:', valid_score_logit)

### Prepare for Catboost

In [0]:
# train_add_features['r_total_towers_killed_thr'] = train_add_features['r_total_towers_killed_thr'].astype(int).astype(str)
train_add_features['d_total_towers_killed_thr'] = train_add_features['d_total_towers_killed_thr'].astype(int).astype(str)
train_add_features['last_tower_radiant'] = train_add_features['last_tower_radiant'].astype(int).astype(str)
# train_add_features['game_time_cat'] = train_add_features['game_time_cat'].astype(str)

# test_ctb['r_total_towers_killed_thr'] = test_ctb['r_total_towers_killed_thr'].astype(int).astype(str)
test_ctb['d_total_towers_killed_thr'] = test_ctb['d_total_towers_killed_thr'].astype(int).astype(str)
test_ctb['last_tower_radiant'] = test_ctb['last_tower_radiant'].astype(int).astype(str)
# test_ctb['game_time_cat'] = test_ctb['game_time_cat'].astype(str)

# train_add_features['game_time_cat'].head()

In [0]:
def make_categorical(feature, df_1=train_add_features, df_2=test_ctb, up_thr=45):
  if df_1.loc[:, feature].dtype == np.bool:
    df_1[feature] = df_1[feature].astype(int).astype(str)
    df_2[feature] = df_2[feature].astype(int).astype(str)
    return
  df_1[feature] = ((df_1[feature] * 100) // 5).astype(int)
  df_2[feature] = ((df_2[feature] * 100) // 5).astype(int)
  df_1[feature] = df_1[feature].map(lambda x: up_thr if x >= up_thr else x).astype(str)
  df_2[feature] = df_2[feature].map(lambda x: up_thr if x >= up_thr else x).astype(str)

In [0]:
features = [
            'total_gold_ratio', 'mean_gold_ratio',
            'total_xp_ratio', 'mean_xp_ratio',
            ] 
for feat in features:
  make_categorical(feat)

In [0]:
for i in range(len(train_add_features.columns)):
  if train_add_features.iloc[:,i].dtype != test_ctb.iloc[:,i].dtype:
    print(train_add_features.columns[i], test_ctb.iloc[:,i].dtype, train_add_features.iloc[:,i].dtype)

In [0]:
cat_ids = list()
for ind, i in enumerate(train_add_features.columns):
  if train_add_features.loc[:,i].dtype != np.int and train_add_features.loc[:,i].dtype != np.float:
    cat_ids.append(ind)
    print(ind, i, train_add_features.loc[:,i].dtype)

0 last_tower_radiant object
207 r_total_towers_killed_thr object
221 kmeans_on_all int32
222 d_total_towers_killed_thr object


In [0]:
train_add_features['last_tower_radiant'] = train_add_features['last_tower_radiant'].astype(int).astype(str)
test_ctb['last_tower_radiant'] = test_ctb['last_tower_radiant'].astype(int).astype(str)

In [0]:
train_add_features['r_total_towers_killed_thr'] = train_add_features['r_total_towers_killed_thr'].astype(int).astype(str)
test_ctb['r_total_towers_killed_thr'] = test_ctb['r_total_towers_killed_thr'].astype(int).astype(str)

### Make class

In [0]:
NUM_CLASSES = 2

class Model:
    def __init__(self, X, y, test, model, cat_ids, model_type, metric, test_size=0.3, random_state=42):
        self.X = X
        self.y = y
        self.test = test
        self.model = model
        self.cat_ids = cat_ids
        self.model_type = model_type
        self.test_size = test_size
        self.metric = metric
        self.random_state = random_state
        self.oof = np.zeros(len(test) * NUM_CLASSES).reshape((len(test), NUM_CLASSES))
        self.kfolds = np.zeros(len(X) * NUM_CLASSES).reshape((len(X), NUM_CLASSES))
        self.error = []
        self.features_importances = np.zeros(X.shape[1])
    
    def _prepareData(self, X, y):
        if self.model_type == 'cat':
            data = Pool(X, y, self.cat_ids)
        elif self.model_type == 'lgb':
            data = Dataset(X, categorical_feature=self.cat_ids, free_raw_data=False)
        return data
    
    def fitModel(self, n_splits=5):
        if self.model_type == 'cat':
            self.model.set_params(cat_features=self.cat_ids)
        elif self.model_type == 'lgb':
            self.model.set_params(categorical_feature=self.cat_ids)
        kfolds = StratifiedKFold(n_splits=5)
        for i, (train_idx, test_idx) in enumerate(kfolds.split(self.X, self.y)):
            X_train, X_test = self.X.values[train_idx], self.X.values[test_idx]
            y_train, y_test = self.y[train_idx], self.y[test_idx]
            print(f'Fold #{i + 1}\n')
            self.model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=500, cat_features=self.cat_ids)
            
            preds_val = self.model.predict_proba(X_test)[:, 1]

            error = self.metric(y_test, preds_val)
            print('eval_metric: ', error)
            self.error.append(error)
            preds_test = self.model.predict_proba(self.test)
            
            self.oof += preds_test
            if self.model_type == 'lgb':
                self.features_importances += self.model.feature_importances_
            elif self.model_type == 'cat':
                self.features_importances += self.model.get_feature_importance()
        self.error = np.array(self.error) / n_splits
        self.oof /= n_splits
        self.features_importances /= n_splits
  
    def getFeaturesImportances(self):
        data = pd.DataFrame(self.features_importances, index=self.X.columns).sort_values(by=0, ascending=False)
        data.columns = ['features_importance']
        return data

    def getError(self):
        return self.error 

    def getOof(self):
        return self.oof

    def getKfoldsPreds(self):
        return self.kfolds

In [0]:
y = target['radiant_win'].astype(int)

In [0]:
dist_y_train = pd.Series(y).value_counts()
params_cat = {
    'task_type': 'GPU',
    'scale_pos_weight': dist_y_train[0] / dist_y_train[1],
    'loss_function': 'Logloss',
    'custom_metric': 'AUC',
    'iterations': 10000,
    'early_stopping_rounds': 200,
    'learning_rate': 0.03,
    'random_seed': 42,
    'bagging_temperature': 0.9,
    'use_best_model': True,
    'ignored_features': ignored_features
}

In [0]:
catboost = CatBoostClassifier(**params_cat)

In [0]:
model = Model(train_add_features, y, test_ctb, catboost, cat_ids, 'cat', roc_auc_score)

In [0]:
model.fitModel()

Fold #1

0:	learn: 0.6823182	test: 0.6820116	best: 0.6820116 (0)	total: 81.5ms	remaining: 13m 35s
500:	learn: 0.4798470	test: 0.4961472	best: 0.4961445 (498)	total: 36.2s	remaining: 11m 27s
1000:	learn: 0.4580736	test: 0.4948480	best: 0.4948480 (1000)	total: 1m 12s	remaining: 10m 49s
1500:	learn: 0.4409942	test: 0.4946984	best: 0.4942640 (1320)	total: 1m 48s	remaining: 10m 12s
bestTest = 0.4942640329
bestIteration = 1320
Shrink model to first 1321 iterations.
eval_metric:  0.8333845669949101
Fold #2

0:	learn: 0.6822277	test: 0.6822046	best: 0.6822046 (0)	total: 82ms	remaining: 13m 40s
500:	learn: 0.4781590	test: 0.5032814	best: 0.5032485 (492)	total: 36.2s	remaining: 11m 26s
bestTest = 0.5021490966
bestIteration = 793
Shrink model to first 794 iterations.
eval_metric:  0.8254222219391736
Fold #3

0:	learn: 0.6820848	test: 0.6819867	best: 0.6819867 (0)	total: 81.5ms	remaining: 13m 35s
500:	learn: 0.4788081	test: 0.4971354	best: 0.4971238 (498)	total: 36.3s	remaining: 11m 27s
1000:	lear

FE for model with aggregate features


In [0]:
feature_imp = model.getFeaturesImportances()
feature_imp[feature_imp['features_importance'] >= 1]

,features_importance
total_gold_ratio,10.856079
mean_gold_ratio,8.828819
total_xp_ratio,4.862657
mean_xp_ratio,4.643231
last_tower_radiant,4.551024
d_total_towers,3.224478
mean_level_ratio,2.458453
std_lh_ratio,2.412914
r_total_towers,2.246544
total_level_ratio,1.714526


FE for model with base features

In [0]:
feature_imp.to_csv('FE_ctb.csv')

In [0]:
pd.options.display.max_rows=220

In [0]:
feature_imp = model.getFeaturesImportances()
print(feature_imp[feature_imp['features_importance'] < 0.20].shape[0])
feature_imp[(feature_imp['features_importance'] < 0.1) & (feature_imp['features_importance'] > 0)].tail(10)

141


,features_importance
r_mean_roshans_killed,0.044461
std_firstblood_claimed_ratio,0.042669
r_total_camps_stacked,0.040142
total_firstblood_claimed_ratio,0.037125
d_mean_sen_placed,0.034219
d_total_obs_placed,0.032568
r_total_obs_placed,0.032548
d_total_firstblood_claimed,0.030691
d_std_firstblood_claimed,0.029912
r_std_firstblood_claimed,0.028330


In [0]:
y_pred_cat = model.getOof()[:,1]
y_pred_cat.shape

(10000,)

In [0]:
print('Validation ROC-AUC score:', model.getError())

Validation ROC-AUC score: [0.16677765 0.16524232 0.1667469  0.16453325 0.16628353]


In [0]:
df_submission_extended = pd.DataFrame(
    {'radiant_win_prob': y_pred_cat}, 
    index=test.index,
)
df_submission_extended.to_csv('submission_test_Catboost_all_without_cells_and_clusters_and_bad_ratio_2.csv')

In [0]:
train_add_features, test_ctb = pd.read_pickle(os.path.join(PATH_FILE, 'train_agg_fin_with_cells.pkl')),\
                               pd.read_pickle(os.path.join(PATH_FILE, 'test_agg_fin_with_cells.pkl'))

bad_cells = [col for col in train_add_features.columns[ignored_features]]

train_add_features['r_total_towers_killed_thr'] = train_add_features['r_total_towers_killed_thr'].astype(int).astype(str)
test_ctb['r_total_towers_killed_thr'] = test_ctb['r_total_towers_killed_thr'].astype(int).astype(str)

train_add_features.drop(columns=bad_cells, inplace=True)
test_ctb.drop(columns=bad_cells, inplace=True)

pd.to_pickle(train_add_features, 'train.pkl')
pd.to_pickle(test_ctb, 'test.pkl')

In [0]:
train_add_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39675 entries, a400b8f29dece5f4d266f49f1ae2e98a to 9928dfde50efcbdb2055da23dcdbc101
Columns: 223 entries, last_tower_radiant to d_total_towers_killed_thr
dtypes: float64(173), int32(1), int64(46), object(3)
memory usage: 67.7+ MB


### Catboost_v2

In [0]:
train['game_time_cat'] = train['game_time'] // 60

In [0]:
 X_train, X_valid, y_train, y_valid = train_test_split(train, 
                                                      target['radiant_win'].astype(int).values, 
                                                      test_size=0.3, 
                                                      random_state=23)

In [0]:
# train['chat_len'] = train['chat_len'] // 60
ids = list(range(5,245, 24))
firstbloods = list(range(23,245,24))
towers = list(range(25,245,24))
roshans = list(range(26,245,24))
obs = list(range(27,245,24))
ids

[5, 29, 53, 77, 101, 125, 149, 173, 197, 221]

In [0]:
train.columns[244]

'd5_sen_placed'

In [0]:
list(enumerate(train.columns[range(5,28)], 5))

In [0]:
x_y_death = list(range(17,245,24)) + list(range(18,245,24)) + list(range(7,245,24))

In [0]:
# ignored = list(set(range(245)) - set(x_y_death))
# len(ignored)

215

In [0]:
categ_feat_idx = [1, 2, 3, 4, 245] + ids + firstbloods + towers + roshans + obs


print(categ_feat_idx)

[1, 2, 3, 4, 245, 5, 29, 53, 77, 101, 125, 149, 173, 197, 221, 23, 47, 71, 95, 119, 143, 167, 191, 215, 239, 25, 49, 73, 97, 121, 145, 169, 193, 217, 241, 26, 50, 74, 98, 122, 146, 170, 194, 218, 242, 27, 51, 75, 99, 123, 147, 171, 195, 219, 243]


In [0]:
ctb = CatBoostClassifier(random_seed=23, 
                         silent=True, 
                         task_type='GPU',
                         border_count=32,
                         ignored_features=ignored
                         )

In [0]:
%%time
ctb.fit(X_train, y_train,);
        # cat_features=categ_feat_idx);

CPU times: user 47.3 s, sys: 14.8 s, total: 1min 2s
Wall time: 53.2 s


In [0]:
y_pred_cat = ctb.predict_proba(X_valid)[:, 1]

In [0]:
valid_score_catboost = roc_auc_score(y_valid, y_pred_cat)
print('Validation ROC-AUC score:', valid_score_catboost)

Validation ROC-AUC score: 0.7812954057293053


In [0]:
y_pred_cat

array([0.36762389, 0.26863302, 0.81018882, ..., 0.48336847, 0.20984568,
       0.57787397])

In [0]:
data = pd.DataFrame(ctb.feature_importances_, index=train.columns).sort_values(by=0, ascending=False)
data.columns = ['features_importance']
data[data['features_importance'] >= 1]

,features_importance
d4_deaths,5.450137
d5_deaths,5.089293
d2_deaths,5.021418
r4_deaths,5.003619
d3_deaths,4.991014
r5_deaths,4.932632
r1_deaths,4.492134
r2_deaths,4.449300
d1_deaths,4.023842
r3_deaths,3.812321


In [0]:
features_have_influence = data[data['features_importance'] >= 1]['features_importance'].index.to_list()
features_have_influence, len(features_have_influence)

(['r5_deaths',
  'r1_deaths',
  'd5_deaths',
  'd4_deaths',
  'd2_deaths',
  'r4_deaths',
  'r1_y',
  'd3_deaths',
  'r3_deaths',
  'r2_deaths',
  'd1_y',
  'r4_y',
  'd4_x',
  'r1_x',
  'r5_x',
  'd1_deaths',
  'd3_hero_id',
  'd1_x',
  'd1_hero_id',
  'r4_x',
  'r2_y',
  'r3_y',
  'r3_hero_id',
  'd4_y',
  'r3_x'],
 25)

#### Logit

In [0]:
%%time
logit = LogisticRegression(random_state=23)
logit.fit(X_train, y_train)

CPU times: user 1min 19s, sys: 140 ms, total: 1min 20s
Wall time: 1min 20s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=23, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_pred_logit = logit.predict_proba(X_valid)[:, 1]

In [0]:
valid_score_logit = roc_auc_score(y_valid, y_pred_logit)
print('Validation ROC-AUC score:', valid_score_logit)

Validation ROC-AUC score: 0.8183336756208583


In [0]:
y_pred_logit

array([0.30130725, 0.55820277, 0.92398193, ..., 0.78556965, 0.30040788,
       0.72219829])

##### Scaling Logit

In [0]:
scaler = StandardScaler()
train_dur_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
test_dur_scaled = scaler.transform(target['radiant_win'].values.reshape(-1, 1))

scalerX = StandardScaler().fit(train)
scalery = StandardScaler().fit(target['radiant_win'].values.reshape(-1,1)) # Have to reshape to avoid warnings

normed_X = scalerX.transform(train)
normed_y = scalery.transform(target['radiant_win'].values.reshape(-1,1)) # Have to reshape to avoid warnings

normed_y = normed_y.ravel() # Turn y back into a vector again

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(normed_X, 
                                                      normed_y, 
                                                      test_size=0.3, 
                                                      random_state=23)

In [0]:
%%time
logit_normed = LogisticRegression(random_state=23)
logit_normed.fit(X_train, y_train.astype('int'))

CPU times: user 4.35 s, sys: 0 ns, total: 4.35 s
Wall time: 4.34 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=23, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_pred_logit_normed = logit.predict_proba(X_valid)[:, 1]

In [0]:
valid_score_logit_normed = roc_auc_score(y_valid.astype('int'), y_pred_logit_normed)
print('Validation ROC-AUC score:', valid_score_logit_normed)

Validation ROC-AUC score: 0.7566220920163143


#### KNN

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
model_knn = KNeighborsClassifier(n_neighbors=(245 // 3))
model_knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=81, p=2,
                     weights='uniform')

In [0]:
y_pred_knn = model_knn.predict_proba(X_valid)[:, 1]

In [0]:
valid_score_knn = roc_auc_score(y_valid.astype('int'), y_pred_knn)
print('Validation ROC-AUC score:', valid_score_knn)

Validation ROC-AUC score: 0.7912607805299894


## blending

In [0]:
logit = pd.read_csv('../submit/submission_test_logit.csv', index_col='match_id_hash')
logit.head()

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.522932
70e5ba30f367cea48793b9003fab9d38,0.948738
4d9ef74d3a2025d79e9423105fd73d41,0.962475
2bb79e0c1eaac1608e5a09c8e0c6a555,0.653978
bec17f099b01d67edc82dfb5ce735a43,0.307651


In [0]:
catboost = pd.read_csv('../submit/submission_test_Catboost.csv', index_col='match_id_hash')
catboost.head()

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.659329
70e5ba30f367cea48793b9003fab9d38,0.973309
4d9ef74d3a2025d79e9423105fd73d41,0.908900
2bb79e0c1eaac1608e5a09c8e0c6a555,0.689098
bec17f099b01d67edc82dfb5ce735a43,0.480375


In [0]:
knn = pd.read_csv('../submit/submission_test_knn.csv', index_col='match_id_hash')
knn.head()

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.629630
70e5ba30f367cea48793b9003fab9d38,0.876543
4d9ef74d3a2025d79e9423105fd73d41,0.851852
2bb79e0c1eaac1608e5a09c8e0c6a555,0.580247
bec17f099b01d67edc82dfb5ce735a43,0.592593


In [0]:
rf = pd.read_csv('../submit/submission_test_RF.csv', index_col='match_id_hash')
rf.head()

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.54
70e5ba30f367cea48793b9003fab9d38,0.78
4d9ef74d3a2025d79e9423105fd73d41,0.87
2bb79e0c1eaac1608e5a09c8e0c6a555,0.58
bec17f099b01d67edc82dfb5ce735a43,0.49


In [0]:
nn = pd.read_csv('../submit/submission_test_nn.csv', index_col='match_id_hash')
nn.head()

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.530560
70e5ba30f367cea48793b9003fab9d38,0.987548
4d9ef74d3a2025d79e9423105fd73d41,0.973294
2bb79e0c1eaac1608e5a09c8e0c6a555,0.579742
bec17f099b01d67edc82dfb5ce735a43,0.289218


In [0]:
blend = 0.3 * logit + 0.25 * nn + 0.22 * catboost + 0.15 * knn + 0.08 * rf
blend.head()

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.572216
70e5ba30f367cea48793b9003fab9d38,0.939518
4d9ef74d3a2025d79e9423105fd73d41,0.929402
2bb79e0c1eaac1608e5a09c8e0c6a555,0.626168
bec17f099b01d67edc82dfb5ce735a43,0.398371


In [0]:
blend.to_csv('submission_test_blending.csv')

Blending score on LB: 0.83123

In [0]:
cat_best = pd.read_csv('/content/cat_boost_w_all_pca_wo_emb.csv', index_col='match_id_hash')
cat_pca = pd.read_csv('/content/cat_boost_w_all_pca.csv', index_col='match_id_hash')
cat_agg = pd.read_csv('/content/submission_test_Catboost_all_without_cells_and_clusters_and_bad_ratio_2.csv', index_col='match_id_hash')

In [0]:
cat_best.head()

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.413273
70e5ba30f367cea48793b9003fab9d38,0.993530
4d9ef74d3a2025d79e9423105fd73d41,0.947810
2bb79e0c1eaac1608e5a09c8e0c6a555,0.855397
bec17f099b01d67edc82dfb5ce735a43,0.264767


In [0]:
cat_pca.head()

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.358641
70e5ba30f367cea48793b9003fab9d38,0.994545
4d9ef74d3a2025d79e9423105fd73d41,0.977063
2bb79e0c1eaac1608e5a09c8e0c6a555,0.869891
bec17f099b01d67edc82dfb5ce735a43,0.226404


In [0]:
cat_agg.head()

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.543050
70e5ba30f367cea48793b9003fab9d38,0.992195
4d9ef74d3a2025d79e9423105fd73d41,0.965062
2bb79e0c1eaac1608e5a09c8e0c6a555,0.770611
bec17f099b01d67edc82dfb5ce735a43,0.351414


In [0]:
blend = 0.7 * cat_best + 0.15 * cat_pca + 0.15 * cat_agg
blend.head()

,radiant_win_prob
match_id_hash,
30cc2d778dca82f2edb568ce9b585caa,0.424545
70e5ba30f367cea48793b9003fab9d38,0.993482
4d9ef74d3a2025d79e9423105fd73d41,0.954786
2bb79e0c1eaac1608e5a09c8e0c6a555,0.844853
bec17f099b01d67edc82dfb5ce735a43,0.272009


In [0]:
blend.to_csv('subm_blend_top_3.csv')

## testing submission

In [0]:
df_submission_extended = pd.DataFrame(
    {'radiant_win_prob': logit.predict_proba(test.values)[:, 1]}, 
    index=test.index,
)
df_submission_extended.to_csv('submission_test_logit_part_data.csv')

logit score on LB: 0.82726

In [0]:
df_submission_extended = pd.DataFrame(
    {'radiant_win_prob': model.predict_proba(test.values)[:, 1]}, 
    index=test.index,
)
df_submission_extended.to_csv('submission_test_RF.csv')

RF score on LB: 0.78605

In [0]:
df_submission_extended = pd.DataFrame(
    {'radiant_win_prob': model_cat.predict_proba(test.values)[:, 1]}, 
    index=test.index,
)
df_submission_extended.to_csv('submission_test_Catboost.csv')

CatBoost score on LB: 0.81483

In [0]:
df_submission_extended = pd.DataFrame(
    {'radiant_win_prob': model_knn.predict_proba(test.values)[:, 1]}, 
    index=test.index,
)
df_submission_extended.to_csv('submission_test_knn.csv')

KNN score on LB: 0.80258